In [128]:
import os
import fitz  # PyMuPDF
import ocrmypdf
import shutil
from pathlib import Path
import pymupdf4llm
import re

# Create output directory
Path("output_markdown").mkdir(parents=True, exist_ok=True)

In [129]:
def is_scanned_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    for page in doc:
        if page.get_text().strip():
            return False
    return True

In [130]:
def run_ocr(input_path, output_path):
    print("🔁 Running OCRmyPDF...")
    ocrmypdf.ocr(
        input_file=input_path,
        output_file=output_path,
        rotate_pages=True,
        deskew=True,
        force_ocr=True,
        skip_text=True
    )
    print(f"✅ OCR complete: {output_path}")

In [131]:
def normalize_heading_hierarchy(md_text):
    lines = md_text.split('\n')
    updated_lines = []
    for line in lines:
        match = re.match(r'^(#+)\s+(.*)', line)
        if match:
            hashes, content = match.groups()
            # Promote/demote based on content
            if len(hashes) == 1:
                hashes = '##'
            elif len(hashes) == 2:
                if any(word in content.lower() for word in ['mortality', 'knock-down', 'resistance', 'vector', 'prevalence']):
                    hashes = '###'
            line = f"{hashes} {content}"
        updated_lines.append(line)
    return '\n'.join(updated_lines)

In [132]:
def extract_markdown_with_hierarchy(pdf_path, md_output_path):
    doc = fitz.open(pdf_path)
    toc = doc.get_toc()
    
    # Set margins to exclude headers/footers (top: 50, bottom: 50 points)
    # This will ignore text in the top 50 points and bottom 50 points of each page
    default_margins = (0, 50, 0, 30)  # (left, top, right, bottom)
    
    if toc:
        # Use table of contents for header detection    
        toc_headers = pymupdf4llm.TocHeaders(doc)
        md_text = pymupdf4llm.to_markdown(
            doc, 
            hdr_info=toc_headers,
            margins=default_margins
        )
        print(f"📋 Used TocHeaders with {len(toc)} TOC entries and margins {default_margins}")
    else:
        # Generate header info with custom settings when no TOC exists
        my_headers = pymupdf4llm.IdentifyHeaders(
            doc, 
            max_levels=3,  # Limit to 3 header levels
            body_limit=11  # Font size limit for body text
        )
        md_text = pymupdf4llm.to_markdown(
            doc, 
            hdr_info=my_headers,
            margins=default_margins
        )
        print("🔍 Used IdentifyHeaders with custom settings and margins")
    
    with open(md_output_path, "w", encoding="utf-8") as f:
        f.write(md_text)
    
    print(f"✅ Markdown with hierarchy and margins saved: {md_output_path}")
    return md_text


In [133]:
def process_pdf_pipeline(pdf_path):
    filename = Path(pdf_path).stem
    output_dir = Path("output_markdown")
    output_dir.mkdir(parents=True, exist_ok=True)

    temp_dir = Path("temp_ocr")
    temp_dir.mkdir(exist_ok=True)

    ocr_path = temp_dir / f"{filename}_ocr.pdf"
    md_output = output_dir / f"{filename}.md"

    print(f"🔍 Processing PDF: {pdf_path}")
    scanned = is_scanned_pdf(pdf_path)

    if scanned:
        print("🧾 Detected scanned PDF")
        run_ocr(pdf_path, ocr_path)
        used_pdf = ocr_path
    else:
        print("📄 Detected born-digital PDF")
        used_pdf = Path(pdf_path)

    return extract_markdown_with_hierarchy(used_pdf, md_output)

In [134]:
test_pdf = "pdfs\Allossogbe_et_al_2017_Mal_J.pdf"
markdown_text = process_pdf_pipeline(test_pdf)

<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\priya\AppData\Local\Temp\ipykernel_30148\2316023380.py:1: SyntaxWarning: invalid escape sequence '\A'
  test_pdf = "pdfs\Allossogbe_et_al_2017_Mal_J.pdf"


🔍 Processing PDF: pdfs\Allossogbe_et_al_2017_Mal_J.pdf
📄 Detected born-digital PDF
📋 Used TocHeaders with 36 TOC entries and margins (0, 50, 0, 30)
✅ Markdown with hierarchy and margins saved: output_markdown\Allossogbe_et_al_2017_Mal_J.md


In [135]:
## DOI if in metadata

doc = fitz.open(test_pdf)
doi = doc.metadata.get("doi", None)
print("DOI:", doi)

DOI: None


In [136]:
## Added DOI if not in metadata

doc = fitz.open(test_pdf)
first_page_text = doc[0].get_text()
match = re.search(r'(10\.\d{4,9}/[-._;()/:A-Z0-9]+)', first_page_text, re.I)
doi = match.group(1) if match else None
print("DOI from text:", doi)


DOI from text: 10.1186/s12936-017-1727-x


In [137]:
def extract_doi(pdf_path: str, bib_entry: dict = None) -> str:
    # 1. Check if provided via BibTeX
    if bib_entry and 'doi' in bib_entry:
        return bib_entry['doi']
    
    # 2. Else extract from PDF text
    doc = fitz.open(pdf_path)
    pages_to_check = [0, len(doc) - 1] if len(doc) > 1 else [0]
    
    for page_num in pages_to_check:
        text = doc[page_num].get_text()
        match = re.search(r'(10\.\d{4,9}/[-._;()/:A-Z0-9]+)', text, re.I)
        if match:
            return match.group(1)
    
    return None  # No DOI found
